# KD線交易策略驗證
Author：余慶龍  
Establish Date：2021.7.25  
Last Modified Date：2021.7.27

## Table of Contents
1. 讀取檔案模塊
2. 指標與交易策略
3. 回測模塊
4. Main：執行上面的模塊

## 1. 讀取檔案模塊

In [5]:
"""
把csv或excel檔案轉換成dataframe
以供後續的pandas操作
"""

import pandas as pd
import numpy as np
from datetime import datetime

def readStock_file(file, filetype='csv'):


    if filetype == 'excel':
        df = pd.read_excel(file, engine='openpyxl', parse_dates=True, header=None)
    else:
        df = pd.read_csv(file)

    # 取代原本的 column 名稱
    # 檔案的日期與開高低收需要照這個順序
    colume_name = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
    df.columns = colume_name

    # 用日期這一行當做 df 的索引
    df = df.set_index('Date')

    # 把日期轉成 datetime的格式(從string)
    df.index = pd.to_datetime(df.index)

    # 照日期排序並把空資料轉成numpy的nan
    df = df.sort_index()
    df = df.replace(r'^\s*-$', np.nan, regex=True)


    for col in df.columns:
        if(col=='Date'):
            continue;
        df[col] = np.array([float(x) for x in df[col]])

    return df

def random_sample(ticker_list_file,  column = '證券代碼', times = 10):
    

    ticker_list_df = pd.read_csv(ticker_list_file, encoding='utf8', usecols=[column])
    ticker_list = ticker_list_df[column].to_list()
    
    sample = np.random.choice(ticker_list, times)
    sample_list = [i.replace(' ', '') for i in sample.tolist()]

    return sample_list

def file_list_with_directory(file_list, path):
    
    files = [path+i+'.csv' for i in file_list]
    return files

def get_dataframe(data_file, days=0):
    ticker = pd.read_csv(data_file)

    # change the name of columns
    ticker.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']

    # set the column "Date" to index
    ticker = ticker.set_index('Date')

    # set index column("Date") from string to datetime.datetime
    ticker.index = pd.to_datetime(ticker.index)

    # sort index column("Date") chronologically
    ticker = ticker.sort_index()
    
    if(days):
        ticker = ticker.tail(days)
    return ticker

## 2. 指標與交易策略

In [6]:
# Define constants
RSI_n = 6
upperBound = 90
lowerBound = 10
middle_line = 50

def kd_indicator(ticker):




    # Create a empty dataFrame with original index
    # To store some temporary data
    tmp = pd.DataFrame(index=ticker.index)

    # Copy a dataFrame to store result
    # deep=True means make a copy instead of link
    data = ticker.copy(deep=True)

    tmp['rise'] = ticker['Close'] - ticker['Open']

    tmp['rise_only'] = tmp['rise']
    tmp['drop_only'] = tmp['rise']

    tmp['rise_only'].loc[tmp['rise'] < 0] = 0
    tmp['drop_only'].loc[tmp['rise'] > 0] = 0


    tmp['rise_avg'] = tmp['rise_only'].rolling(RSI_n, min_periods=1).mean()
    tmp['drop_avg'] = tmp['drop_only'].rolling(RSI_n, min_periods=1).mean().abs()


    data['RSI'] = tmp['rise_avg'] / (tmp['rise_avg'] + tmp['drop_avg']) * 100
    
    return data

In [7]:
def plot_trading(data, bound='normal'):
    
    plots = []
    
    # the position in graph
    buy, sell = [], []

    # Adjust the position to avoid signals block the lines
    for index, row in data.iterrows():
        buy.append(row['Close']*0.985 if row['buy'] else np.nan)
        sell.append(row['Close']*1.01 if row['sell'] else np.nan)


    plots = [ 
        # Add the KD indicator to plot
        #     '#c8d164' is yellow
        #     '#7dc2fa' is blue
        mpf.make_addplot(data['k'], panel = 1, color='#c8d164', title='K(yellow), D(blue)'),
        mpf.make_addplot(data['d'], panel = 1, color='#7dc2fa'),
    ]
    
    # Add upperbound & lowerbound to plot
    if bound == 'normal':
        plots += [
            mpf.make_addplot(np.full((len(data['k'])), upperBound), panel = 1, color='#bf5ce0', alpha=0.5),
            mpf.make_addplot(np.full((len(data['d'])), lowerBound), panel = 1, color='#7e3696', alpha=0.5),
        ]
    elif bound == 'middle':
        plots += [
            mpf.make_addplot(np.full((len(data['k'])), middleBound), panel = 1, color='#bf5ce0', alpha=0.5),
        ]  

    # Add trading point to plot
    #    np.isnan(sell) determine the value is nan or not
    #    count_nonzero(np.isnan(sell)) counts how many nan we have
    #    if nan amount equal the length of array, we don't have sell record
    if not np.count_nonzero(np.isnan(sell)) == len(sell):
        plots.append(mpf.make_addplot(sell , type = 'scatter', color = 'red', marker = 'v', markersize = 100))
    if not np.count_nonzero(np.isnan(buy)) == len(buy):
        plots.append(mpf.make_addplot(buy , type = 'scatter', color = '#cfc01d', marker = '^', markersize = 200))

    mpf.plot( data, 
                type='candle',       # candlestick chart of stock price
                volume=True,         # plots trading volume as well
                volume_panel=2,
                figscale=2,          # make graph twice bigger
                style='yahoo',       # color scheme using 'yahoo' style
                addplot = plots,
    )    

In [15]:
import mplfinance as mpf

def bound(ticker, plot=True):
    

    upperbound = 95
    lowerbound = 15

    # skip first RSI_n days
    for i in range(RSI_n, data.shape[0]):

        if(data['RSI'][i] < lowerbound):


            data.loc[(data.index[i], 'buy')] = 1
            print('buy  |', data.index[i], data['Close'][i])
        if(data['RSI'][i] > upperbound):

            data.loc[(data.index[i], 'sell')] = 1
            print('sell |', data.index[i], data['Close'][i])
            
    if plot:
        plot_trading(data) 
    return data

def stagnant(ticker, plot=True):
   
    middle_line = 50
    trading_interval = 6
    trading_cooldown = trading_interval
    stagnant_upper, stagnant_lower = 0, 0


    # skip first RSI_n days
    for i in range(RSI_n, data.shape[0]):

        trading_cooldown += 1
        # 在 upperbound 以上的話， counter就加一
        # 否則counter歸零
        if(data['RSI'][i] >= middle_line):
            stagnant_upper += 1
        else:
            stagnant_upper = 0

        # 在 lowerbound 以下的話， counter就加一
        # 否則counter歸零
        if(data['RSI'][i] <= middle_line ):
            stagnant_lower += 1
        else:
            stagnant_lower = 0

        if(stagnant_upper > trading_interval and trading_cooldown > trading_interval):
            trading_cooldown = 0
            data.loc[(data.index[i], 'buy')] = 1
            print('buy  |', data.index[i], data['Close'][i])

        if(stagnant_lower > trading_interval and trading_cooldown > trading_interval):
            trading_cooldown = 0
            data.loc[(data.index[i], 'sell')] = 1
            print('sell |', data.index[i], data['Close'][i])

    
    if plot:
        plot_trading(data, bound='middle') 
    return data

## 3. 回測模塊

In [9]:
def backtesting(data):
    # how many shares we hold, and the money balance
    have_shares , balance = 0, 0

    # multiple buy and 1 sell as "a trade"
    # record trading times and how many trade are making money
    make_money, trade_times = 0, 0

    # we use geometic average
    # this is products of return rate
    return_rate = 1
    for index, row in data.iterrows():

        if(row['buy'] == 1):
            balance -= row['Close']
            have_shares += 1
            # print('buy ', row['Close'])

        # sell signal and we have some shares
        elif(row['sell'] == 1 and have_shares != 0):


            # print('sell', row['Close'], "*" , have_shares)

            principal = -balance
            balance += row['Close'] * have_shares

            # to calculate accuracy
            trade_times += 1
            if(balance > 0):
                make_money += 1

            # plus 1 avoiding get too small
            return_rate *= 1+(balance / principal)

            # reset balance & shares
            balance = 0
            have_shares = 0


    return_rate = (return_rate-1) * 100.0
    
    accuracy = 0.0
    if(trade_times):
        accuracy = make_money/ trade_times * 100

    return return_rate, accuracy

## 4. Main：執行上面的模塊

In [18]:
random_choice_num = 5
plot_trading = False
data_directory = '../../parse_data/data/splited/'
ticker_name_file=  '../../parse_data/data/ticker_name.csv'


strategy_list = [bound, stagnant]
dictionary_rr = {}
dictionary_acc = {}

for data_file in file_list_with_directory(random_sample(ticker_list_file = ticker_name_file, times = random_choice_num), data_directory):
    
    slim_file_name = (data_file.split('/')[-1]).split('.')[0]

    print(data_file)
    ticker = get_dataframe(data_file, days=200)
    ticker = kd_indicator(ticker)
    
    strategy_rr= []
    strategy_acc = []
    for i in range(len(strategy_list)):
        
        data = ticker.copy(deep=True)
        
        # initialize 'buy' & 'sell' column by filling with 0
        data['buy'] = np.zeros(ticker.shape[0])
        data['sell'] = np.zeros(ticker.shape[0])

        data = strategy_list[i](data, plot_trading)
        return_rate, acc = backtesting(data)
        
        return_rate = (round(return_rate, 1))
        acc = (round(acc, 3))
        print('\treturn rate: {:>5.1f}% | acc: {:>5.1f}% | {}'.format(return_rate, acc, strategy_list[i].__name__))
        strategy_rr.append(str(return_rate)+'%')
        strategy_acc.append(str(acc)+'%')
    

    #print(strategy_rr)
    #print(strategy_acc)
    
    dictionary_rr[slim_file_name] = strategy_rr
    dictionary_acc[slim_file_name] = strategy_acc


strategy_names = [i.__name__ for i in strategy_list]
dataframe_return_rate = pd.DataFrame.from_dict(dictionary_rr, orient='index',columns = strategy_names)
dataframe_acc = pd.DataFrame.from_dict(dictionary_acc ,orient='index',columns = strategy_names)

print()
print('return rate:')
print(dataframe_return_rate)

print()
print('accuracy:')
print(dataframe_acc)


../../parse_data/data/splited/3708.csv
buy  | 2020-06-10 00:00:00 85.8
buy  | 2020-06-11 00:00:00 83.2
buy  | 2020-07-28 00:00:00 92.0
buy  | 2020-08-07 00:00:00 104.0
buy  | 2020-08-10 00:00:00 105.5
buy  | 2020-08-11 00:00:00 103.0
buy  | 2020-08-12 00:00:00 100.0
buy  | 2020-08-13 00:00:00 97.6
buy  | 2020-08-14 00:00:00 98.2
buy  | 2020-10-05 00:00:00 144.0
buy  | 2020-10-22 00:00:00 157.5
buy  | 2020-11-17 00:00:00 161.0
buy  | 2020-12-03 00:00:00 163.5
buy  | 2020-12-04 00:00:00 162.0
buy  | 2020-12-07 00:00:00 157.0
buy  | 2020-12-08 00:00:00 156.5
buy  | 2020-12-17 00:00:00 144.5
buy  | 2020-12-18 00:00:00 145.5
buy  | 2021-01-15 00:00:00 134.5
buy  | 2021-03-24 00:00:00 135.5
buy  | 2021-03-25 00:00:00 135.0
buy  | 2021-03-26 00:00:00 136.0
	return rate:   0.0% | acc:   0.0% | bound
sell | 2020-06-18 00:00:00 84.9
sell | 2020-07-24 00:00:00 99.3
sell | 2020-08-14 00:00:00 98.2
buy  | 2020-08-31 00:00:00 142.0
sell | 2020-10-06 00:00:00 158.0
buy  | 2020-10-16 00:00:00 166.0
se